Connect to Kaggle

In [0]:
# see more details in the course first lecture
!mkdir /root/.kaggle/
import json
import os

# Installing the Kaggle package
!pip install kaggle 

#Important Note: complete this with your own key - after running this for the first time remmember to **remove** your API_KEY
api_token = {"username":"","key":""}  # TODO: REMOVE THIS LINE!!!

# creating kaggle.json file with the personal API-Key details 
# You can also put this file on your Google Drive
with open('/root/.kaggle/kaggle.json', 'w') as file:
  json.dump(api_token, file)
!chmod 600 /root/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle/’: File exists


Search for the Corona Virus Dataset

In [0]:
!rm -r ./datasets
# searching for the dataset
DATASET_NAME = "corona"
DATSET_KAGGLE = "sudalairajkumar/novel-corona-virus-2019-dataset"
!mkdir -p ./datasets/$DATASET_NAME/data
!ls datasets/$DATASET_NAME/data

!kaggle datasets list -s $DATSET_KAGGLE

ref                                                              title                                               size  lastUpdated          downloadCount  
---------------------------------------------------------------  -------------------------------------------------  -----  -------------------  -------------  
unanimad/corona-virus-brazil                                     Coronavirus - Brazil                                13KB  2020-04-08 20:43:56           6003  
sudalairajkumar/novel-corona-virus-2019-dataset                  Novel Corona Virus 2019 Dataset                    706KB  2020-04-09 06:36:40         129782  
ryanxjhan/coronaviruscovid19-canada                              Coronavirus/COVID-19 Canada                        147KB  2020-04-04 21:56:01            130  
ardisragen/indonesia-coronavirus-cases                           Indonesia-Coronavirus                                9KB  2020-04-08 16:53:53            991  
gkhan496/covid19-in-turkey              

Download Corona Virus using Kaggle

In [0]:
# download the dataset from Kaggle and unzip it
!kaggle datasets download $DATSET_KAGGLE -p ./datasets/$DATASET_NAME/zip
!unzip ./datasets/$DATASET_NAME/zip/*.zip -d ./datasets/$DATASET_NAME/data
!ls ./datasets/$DATASET_NAME/zip/
!ls ./datasets/$DATASET_NAME/data/

  0% 0.00/706k [00:00<?, ?B/s]
100% 706k/706k [00:00<00:00, 96.3MB/s]
Archive:  ./datasets/corona/zip/novel-corona-virus-2019-dataset.zip
  inflating: ./datasets/corona/data/COVID19_line_list_data.csv  
  inflating: ./datasets/corona/data/COVID19_open_line_list.csv  
  inflating: ./datasets/corona/data/covid_19_data.csv  
  inflating: ./datasets/corona/data/time_series_covid_19_confirmed.csv  
  inflating: ./datasets/corona/data/time_series_covid_19_confirmed_US.csv  
  inflating: ./datasets/corona/data/time_series_covid_19_deaths.csv  
  inflating: ./datasets/corona/data/time_series_covid_19_deaths_US.csv  
  inflating: ./datasets/corona/data/time_series_covid_19_recovered.csv  
novel-corona-virus-2019-dataset.zip
covid_19_data.csv		     time_series_covid_19_confirmed_US.csv
COVID19_line_list_data.csv	     time_series_covid_19_deaths.csv
COVID19_open_line_list.csv	     time_series_covid_19_deaths_US.csv
novel-corona-virus-2019-dataset.zip  time_series_covid_19_recovered.csv
time_serie

Read using Pandas


In [0]:
import pandas as pd 
# Read Data
file_of_intrest = 'covid_19_data.csv'
DATASET_PATH = f'datasets/{DATASET_NAME}/data/{file_of_intrest}'
df = pd.read_csv(DATASET_PATH)
# Rename columns
df = df.rename(columns={'Country/Region':'Country'})
df = df.rename(columns={'ObservationDate':'Date'})
df

,SNo,Date,Province/State,Country,Last Update,Confirmed,Deaths,Recovered,Month,Day,Year
0,1,01/22/2020,Anhui,Mainland China,1/22/2020 17:00,1.0,0.0,0.0,01,22,2020
1,2,01/22/2020,Beijing,Mainland China,1/22/2020 17:00,14.0,0.0,0.0,01,22,2020
2,3,01/22/2020,Chongqing,Mainland China,1/22/2020 17:00,6.0,0.0,0.0,01,22,2020
3,4,01/22/2020,Fujian,Mainland China,1/22/2020 17:00,1.0,0.0,0.0,01,22,2020
4,5,01/22/2020,Gansu,Mainland China,1/22/2020 17:00,0.0,0.0,0.0,01,22,2020
...,...,...,...,...,...,...,...,...,...,...,...
13204,13205,04/08/2020,Wyoming,US,2020-04-08 22:59:00,221.0,0.0,0.0,04,08,2020
13205,13206,04/08/2020,Xinjiang,Mainland China,2020-04-08 22:59:00,76.0,3.0,73.0,04,08,2020
13206,13207,04/08/2020,Yukon,Canada,2020-04-08 22:59:00,7.0,0.0,0.0,04,08,2020
13207,13208,04/08/2020,Yunnan,Mainland China,2020-04-08 22:59:00,184.0,2.0,173.0,04,08,2020


In [0]:
# Group by Country and Date
df_countries = df.groupby(['Country', 'Date']).sum().reset_index().sort_values('Date', ascending=False)
df_countries

,Country,Date,SNo,Confirmed,Deaths,Recovered
3474,Latvia,04/08/2020,12981,577.0,2.0,16.0
1076,Cambodia,04/08/2020,12919,117.0,0.0,63.0
546,Bangladesh,04/08/2020,12902,218.0,20.0,33.0
4588,Nigeria,04/08/2020,13010,276.0,6.0,44.0
1675,Dominican Republic,04/08/2020,12936,2111.0,108.0,50.0
...,...,...,...,...,...,...
2581,Hong Kong,01/22/2020,13,0.0,0.0,0.0
3156,Japan,01/22/2020,36,2.0,0.0,0.0
5747,South Korea,01/22/2020,38,1.0,0.0,0.0
6558,US,01/22/2020,32,1.0,0.0,0.0


In [0]:
# Import libraries
import plotly.graph_objs as go


# Create the Choropleth
column_of_interst = 'Deaths'
date_of_interst = '04/06/2020'
df_countries_date = df_countries.copy()
df_countries_date = df_countries_date[df_countries_date[column_of_interst]>0]
df_countries_date.loc['Date'] = pd.to_datetime(df_countries_date['Date'], format="%m/%d/%Y")
df_countries_date = df_countries_date[df_countries_date['Date'] == date_of_interst]
fig = go.Figure(data=go.Choropleth(
    locations = df_countries_date['Country'],
    locationmode = 'country names',
    z = df_countries_date[column_of_interst],
    colorscale = 'Reds',
    marker_line_color = 'black',
    marker_line_width = 0.5,
))
fig.update_layout(
    title_text = f'{column_of_interst} Cases of {date_of_interst}',
    title_x = 0.5,
    geo=dict(
        showframe = False,
        showcoastlines = False,
        projection_type = 'equirectangular'
    )
)

In [0]:
import plotly.express as px

# Manipulating the original dataframe
column_of_interst = 'Deaths'
df_countries_date = df_countries.copy()
df_countries_date = df_countries_date[df_countries_date[column_of_interst]>0]
df_countries_date = df_countries_date.sort_values('Date', ascending=True)
# Creating the visualization
fig = px.choropleth(df_countries_date, 
                    locations="Country", 
                    locationmode = "country names",
                    color=column_of_interst,
                    hover_name="Country", 
                    animation_frame="Date",
                    color_continuous_scale='Reds'
                   )
fig.update_layout(
    title_text = f'Global {column_of_interst} Over Time',
    title_x = 0.5,
    geo=dict(
        showframe = False,
        showcoastlines = False,
    ))
    
fig.show()